In [125]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
from sklearn.neighbors import NearestNeighbors
import random
import math

In [126]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8294 entries, 0 to 8293
Data columns (total 24 columns):
Customer                         8294 non-null object
State                            8294 non-null object
Customer Lifetime Value          8294 non-null float64
Response                         8294 non-null object
Coverage                         8294 non-null object
Education                        8294 non-null object
Effective To Date                8294 non-null object
EmploymentStatus                 8293 non-null object
Gender                           8293 non-null object
Income                           8293 non-null float64
Location Code                    8293 non-null object
Marital Status                   8293 non-null object
Monthly Premium Auto             8293 non-null float64
Months Since Last Claim          8293 non-null float64
Months Since Policy Inception    8293 non-null float64
Number of Open Complaints        8293 non-null float64
Number of Policies     

In [127]:
full = pd.read_excel("Round2_Dataset_G1.xlsx")
data = full.loc[full["Response"]=="Yes",]

In [128]:
cont = data.describe(0).columns
cat = data.select_dtypes([object]).columns
median = np.median(np.std(data[cont]))
cat2 = cat.drop(["Effective To Date","Customer"])

In [129]:
final = data
for i in cat.drop(["Effective To Date","Customer"])[1:]:
    final = pd.concat([final,median*pd.get_dummies(data[i],drop_first=True)],axis=1)
final.reset_index(inplace=True,drop=True)
final.drop(cat,axis=1,inplace=True)

In [130]:
from sklearn.neighbors import NearestNeighbors
def newnearest(x):
    nbrs = NearestNeighbors(n_neighbors=6,algorithm="ball_tree",leaf_size=100,metric="euclidean")
    nbrs.fit(x)
    return nbrs.kneighbors(x,return_distance=False)

In [131]:
index = newnearest(final)
index = index.T[1:].T

In [134]:
def synthetic(n,sample,t):
    global index
    global data
    global full
    if n<100:
        t = int(t*(n/100))
        n=100
    n=n//100    
    sample = sample.head(t)
    it=0
    
    select = [(random.sample(list(value),2)) for value in index]
    for i,x in zip(sample.iterrows(),select):
        dic = {}
    
    
        for y in x:
            k=0
            gap = random.randrange(0,10)/10
            dic = dict(gap*(i[1][cont]-sample[cont].iloc[y]) +i[1][cont])
    
    
            ind = data.iloc[index[it]][cat2].mode().iloc[0].T.index
            val = data.iloc[index[it]][cat2].mode().iloc[0].T.values      
            for l,m in zip(ind,val):
                try:
                    if math.isnan(m):
                        m = i[1][l]
                except TypeError:
                    pass
                finally:
                    dic[l]=m
            
            dic["Customer"] = "SYN" + str(k) + str(it)
            dic["Effective To Date"] = data.iloc[it]["Effective To Date"]
            k+=1
            full = full.append(Series(dic),ignore_index=True)
        
        it += 1
        
        

In [135]:
t=final.shape[0]
n = 200
send_data= data.drop(["Effective To Date","Customer"],axis=1).reset_index(drop=True)
synthetic(n,send_data,t)

In [136]:
full.apply(lambda x: sum(x.isnull()))

Customer                         0
State                            0
Customer Lifetime Value          0
Response                         0
Coverage                         0
Education                        0
Effective To Date                0
EmploymentStatus                 1
Gender                           1
Income                           1
Location Code                    1
Marital Status                   1
Monthly Premium Auto             1
Months Since Last Claim          1
Months Since Policy Inception    1
Number of Open Complaints        1
Number of Policies               1
Policy Type                      1
Policy                           1
Renew Offer Type                 1
Sales Channel                    1
Total Claim Amount               1
Vehicle Class                    1
Vehicle Size                     1
dtype: int64

In [137]:
full.to_excel("Round2_Output_G1_OsheenNayak.xlsx",index=False)